In [29]:
!pwd
import sys
sys.executable

/c/Users/shingo/jupyter_notebook/tfgpu_py36_work/02_keras_py/experiment/01_code_test


'C:\\Users\\shingo\\Anaconda3\\envs\\tfgpu20\\python.exe'

# Focal Loss test


In [2]:
import os, sys
gpu_num = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

# 自作モジュールimport
#sys.path.append('/gpfsx01/home/aaa00162/jupyterhub/notebook/H3-057/Submit/make_hERG_PAMPA_model/make_model/')
#from modules import *
#sys.path.append('/gpfsx01/home/aaa00162/jupyterhub/notebook/H3-057/Submit/make_hERG_PAMPA_model/make_model/modules/Git/mixup-generator/')
#from random_eraser import get_random_eraser
%reload_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
#import keras.backend as K
keras.backend.clear_session()
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

## Multi-class binary, categoricalの時のFocal Loss
- https://github.com/umbertogriffo/focal-loss-keras/blob/master/losses.py

In [3]:
import dill

def binary_focal_loss(gamma=2., alpha=.25, mask_value=None):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
     
     # モデルロード時は以下のようにcustom_objectが必要
     import dill
     custom_object = {'binary_focal_loss_fixed': dill.loads(dill.dumps(binary_focal_loss(gamma=2., alpha=.25)))
                      , 'categorical_focal_loss_fixed': dill.loads(dill.dumps(categorical_focal_loss(gamma=2., alpha=.25)))
                      , 'categorical_focal_loss': categorical_focal_loss
                      , 'binary_focal_loss': binary_focal_loss}
      model = keras.models.load_model(input_model_path, custom_objects=custom_object)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        # 欠損ラベル指定あれば取り除く
        if mask_value is not None:
            mask = keras.backend.cast(keras.backend.not_equal(y_true, mask_value), keras.backend.floatx())
            y_true = y_true * mask
            y_pred = y_pred * mask
        
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = keras.backend.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = keras.backend.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = keras.backend.clip(pt_0, epsilon, 1. - epsilon)

        return -keras.backend.sum(alpha * keras.backend.pow(1. - pt_1, gamma) * keras.backend.log(pt_1)) \
               -keras.backend.sum((1 - alpha) * keras.backend.pow(pt_0, gamma) * keras.backend.log(1. - pt_0))

    return binary_focal_loss_fixed


def categorical_focal_loss(gamma=2., alpha=.25, mask_value=None):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
     
     # モデルロード時は以下のようにcustom_objectが必要
     import dill
     custom_object = {'binary_focal_loss_fixed': dill.loads(dill.dumps(binary_focal_loss(gamma=2., alpha=.25)))
                      , 'categorical_focal_loss_fixed': dill.loads(dill.dumps(categorical_focal_loss(gamma=2., alpha=.25)))
                      , 'categorical_focal_loss': categorical_focal_loss
                      , 'binary_focal_loss': binary_focal_loss}
      model = keras.models.load_model(input_model_path, custom_objects=custom_object)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """
        # 欠損ラベル指定あれば取り除く
        if mask_value is not None:
            mask = keras.backend.cast(keras.backend.not_equal(y_true, mask_value), keras.backend.floatx())
            y_true = y_true * mask
            y_pred = y_pred * mask
        
        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= keras.backend.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = keras.backend.epsilon()
        y_pred = keras.backend.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * keras.backend.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * keras.backend.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return keras.backend.sum(loss, axis=1)

    return categorical_focal_loss_fixed

## 3node learning Demo
- https://www.dlology.com/blog/how-to-multi-task-learning-with-missing-labels-in-keras/


- 出力層3nodeの場合

In [4]:
def multi_learning_demo(activation='sigmoid', loss='binary_crossentropy', mask_value=None):
    """
    https://www.dlology.com/blog/how-to-multi-task-learning-with-missing-labels-in-keras/
    """
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.model_selection  import train_test_split
    import tensorflow as tf
    import numpy as np
    seed = 42
    np.random.seed(seed) # 乱数シード固定
    
    # 2D空間で100,000個のデータポイントをランダムに生成します。各軸の範囲は0〜1
    N =100000
    X = np.random.rand(N,2)
    
    threshold = 0.5
    if isinstance(loss, str) == False:
        # focal loss用に不均衡にする
        threshold = 0.05
    
    # place holder for Y
    Y = np.ones((N,3))
    if activation == 'sigmoid':
        print('sigmoid')
        Y[:,0] = X[:, 1] <= threshold
        Y[:,1] = X[:, 0] >= threshold
        Y[:,2] = X[:, 0] + X[:, 1] > 1
    else:
        print('not sigmoid')
        Y[:,0] = X[:, 1] <= threshold
        Y[:,1] = X[:, 1] > threshold
        Y[:,2] = 0

    # Mask for missing label.
    if mask_value is not None:
        # Drop 2% y0.
        Y[: int(N*0.020), 0] = mask_value
        # Drop 0.7% y1.
        Y[int(N*0.018): int(N*0.025), 1] = mask_value
        # Drop 1.1% y2.
        Y[int(N*0.024): int(N*0.035), 2] = mask_value

    print(Y.shape, Y)
    for i in range(3): 
        print(pd.Series(Y[:,i]).value_counts())
        
    X_train, X_dev, y_train, y_dev = train_test_split(X[:-3000], Y[:-3000], test_size=0.9, random_state=seed)

    keras.backend.clear_session()
    model = keras.models.Sequential()

    input_dim = X_train.shape[1]
    nb_classes = y_train.shape[1]
    print('input_dim, nb_classes:', input_dim, nb_classes)

    model.add(keras.layers.Dense(10, input_dim=input_dim, activation='relu', name='input'))
    model.add(keras.layers.Dense(20, activation='relu', name='fc1'))
    model.add(keras.layers.Dense(10, activation='relu', name='fc2'))
    model.add(keras.layers.Dense(nb_classes, activation=activation, name='output'))

    model.compile(loss=loss, optimizer='nadam', metrics=['accuracy'])

    history = model.fit(X_train, y_train
                        , validation_data=(X_dev,y_dev)
                        , epochs=300
                        , batch_size=500000, verbose = 0)

    print(model.metrics_names, model.evaluate(X[-3000:],Y[-3000:]))
    
    return model

#### sigmoid + Focal Lossなし
- うまくいってる

In [5]:
model = multi_learning_demo(activation='sigmoid', loss='binary_crossentropy')
x = np.random.rand(1,2)
print(x, model.predict(x))

sigmoid
(100000, 3) [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 0.]
 ...
 [0. 0. 0.]
 [1. 1. 0.]
 [1. 1. 0.]]
0.0    50159
1.0    49841
dtype: int64
1.0    50073
0.0    49927
dtype: int64
1.0    50034
0.0    49966
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.135534831960996, 0.9824446]
[[0.28258797 0.45867659]] [[0.58103794 0.03488062 0.07242857]]


#### sigmoid + binary_Focal_Lossあり
- なんかおかしい。3nodeだからか精度は下がってる（sigmoidだから本来は1nodeでやるべき）

In [6]:
model = multi_learning_demo(activation='sigmoid'
                            , loss=[binary_focal_loss(alpha=.25, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

sigmoid
(100000, 3) [[0. 1. 1.]
 [0. 1. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95109
0.0     4891
dtype: int64
1.0    50034
0.0    49966
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [1.324547742207845, 0.91]
[[0.28258797 0.45867659]] [[0.16331042 0.7137602  0.21853684]]


#### sigmoid +  欠損ラベル作成して、maskありでbinary_Focal_Loss使用
- なんかおかしい。3nodeだからか精度は下がってる（sigmoidだから本来は1nodeでやるべき）

In [7]:
mask_value = -1.0
model = multi_learning_demo(activation='sigmoid'
                            , loss=[binary_focal_loss(alpha=.25, gamma=2, mask_value=mask_value)]
                            , mask_value=mask_value)
x = np.random.rand(1,2)
print(x, model.predict(x))

sigmoid
(100000, 3) [[-1.  1.  1.]
 [-1.  1.  1.]
 [-1.  1.  0.]
 ...
 [ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  1.  0.]]
 0.0    93132
 1.0     4868
-1.0     2000
dtype: int64
 1.0    94440
 0.0     4859
-1.0      701
dtype: int64
 1.0    49495
 0.0    49405
-1.0     1100
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [1.6527171522776285, 0.91]
[[0.28258797 0.45867659]] [[0.14916639 0.72698677 0.2608265 ]]


#### softmax + Focal Lossなし
- うまくいってる

In [8]:
model = multi_learning_demo(activation='softmax'
                            , loss='categorical_crossentropy')
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
0.0    50159
1.0    49841
dtype: int64
1.0    50159
0.0    49841
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.08374507993459701, 0.995]
[[0.28258797 0.45867659]] [[0.67203224 0.32321072 0.00475709]]


In [9]:
# gamma=0はfocal_loss無効化(=Cross Entropy Lossと一致)したとき
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=1., gamma=0)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.06129170957704385, 0.982]
[[0.28258797 0.45867659]] [[0.00169201 0.9960477  0.0022603 ]]


#### softmax + categorical_Focal_Lossあり
- うまくいってて、精度上がってる（不均衡そこまで強くないから精度下がるときもあり）


#### gamma(=より難しい focus すべき example が学習に強く寄与させるパラメータ)変えて試す

In [10]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=1)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.006908638485862563, 0.98933333]
[[0.28258797 0.45867659]] [[0.02710329 0.96970606 0.00319058]]


In [11]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.004425893266374866, 0.97433335]
[[0.28258797 0.45867659]] [[0.07611087 0.9017847  0.02210436]]


In [12]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=3)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.0022603856835824747, 0.98366666]
[[0.28258797 0.45867659]] [[0.05019373 0.88784873 0.06195754]]


In [13]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=4)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.0024896944255257646, 0.948]
[[0.28258797 0.45867659]] [[0.27427882 0.7020251  0.02369612]]


In [14]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=5)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.0010318932885614535, 0.948]
[[0.28258797 0.45867659]] [[0.21530646 0.7135891  0.07110445]]


#### alpha(=クラスアンバランスを補正するハイパーパラメタ。クラス間の重み付けで、classes_weightに関連するパラメ)変えて試す

In [15]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.15, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.002379781760275364, 0.9866667]
[[0.28258797 0.45867659]] [[0.08222488 0.90650374 0.01127141]]


In [16]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.004964385270451506, 0.95666665]
[[0.28258797 0.45867659]] [[0.05555156 0.93470895 0.00973954]]


In [17]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.35, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.005458291701041162, 0.98333335]
[[0.28258797 0.45867659]] [[0.04404321 0.93573105 0.02022573]]


In [18]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.45, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.012044737313563625, 0.948]
[[0.28258797 0.45867659]] [[0.07506086 0.91323495 0.01170424]]


In [19]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.55, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.016282597569127876, 0.948]
[[0.28258797 0.45867659]] [[0.12402618 0.8667344  0.00923939]]


In [20]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.65, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.028358600248893102, 0.948]
[[0.28258797 0.45867659]] [[0.16653135 0.8230371  0.01043157]]


In [21]:
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.75, gamma=2)])
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95031
0.0     4969
dtype: int64
0.0    100000
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.03285398528476556, 0.948]
[[0.28258797 0.45867659]] [[0.12683812 0.8592996  0.01386235]]


#### softmax + 欠損ラベル作成して、maskありでcategorical_Focal_Loss使用
- うまくいってる

In [22]:
mask_value = -1.0
model = multi_learning_demo(activation='softmax'
                            , loss=[categorical_focal_loss(alpha=.25, gamma=2, mask_value=mask_value)]
                            , mask_value=mask_value)
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[-1.  1.  0.]
 [-1.  1.  0.]
 [-1.  1.  0.]
 ...
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]]
 0.0    93132
 1.0     4868
-1.0     2000
dtype: int64
 1.0    94365
 0.0     4934
-1.0      701
dtype: int64
 0.0    98900
-1.0     1100
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.0046465199664235115, 0.9723333]
[[0.28258797 0.45867659]] [[0.05798003 0.9214106  0.02060932]]


### マルチラベル+cross_entropy_lossの時のFocalLoss

In [23]:
def build_masked_loss(loss_function, mask_value=-1):
    """
    ラベル付けされたデータを「マスクする」損失関数
    https://www.dlology.com/blog/how-to-multi-task-learning-with-missing-labels-in-keras/
    https://github.com/keras-team/keras/issues/3893
    マルチラベル（多クラス多ラベル分類）は sigmoid+binary_crossentropy
    Args:
        loss_func: The loss function to mask（K.binary_crossentropy or K.categorical_crossentropy）
        mask_value: The value to mask in the targets（欠損ラベルとしてマスクする値）
    Returns:
        function: a loss function that acts like loss_function with masked inputs
    """
    def masked_loss_function(y_true, y_pred):
        mask = keras.backend.cast(keras.backend.not_equal(y_true, mask_value), keras.backend.floatx())
        # 欠損の要素が削除される。テンソルのshapeが[batch_size, クラス数+1(欠損クラス)]→[batch_size, クラス数]になる
        return loss_function(y_true * mask, y_pred * mask)
    return masked_loss_function

class FocalLoss_multilabel(object):
    """
    マルチラベル+cross_entropy_lossの時のFocalLoss
    https://github.com/keras-team/keras/issues/10371
    loss関数を引数で渡す様にした

    FocalLossの論文解説記事：
    https://towardsdatascience.com/handling-class-imbalanced-data-using-a-loss-specifically-made-for-it-6e58fd65ffab
    論文ではCIFAR10のクラス不均衡が100倍、200倍の時focal loss有効だった

    Usage:
        f_loss = multi_loss.FocalLoss(loss_function=multi_loss.build_masked_loss(K.binary_crossentropy)).compute_loss
        model.compile(loss=f_loss, optimizer='nadam', metrics=['accuracy', masked_accuracy])
        model.fit_generator(…)
    """
    def __init__(self, gamma=2, alpha=0.25, loss_function=keras.backend.binary_crossentropy):
        self._gamma = gamma
        self._alpha = alpha
        self._loss_function = loss_function
        
    def compute_loss(self, y_true, y_pred):
        cross_entropy_loss = self._loss_function(y_true, y_pred)
        p_t = ((y_true * y_pred) + ((1 - y_true) * (1 - y_pred)))
        modulating_factor = 1.0
        if self._gamma:
            modulating_factor = tf.pow(1.0 - p_t, self._gamma)
        alpha_weight_factor = 1.0
        if self._alpha is not None:
            alpha_weight_factor = (y_true * self._alpha + (1 - y_true) * (1 - self._alpha))
        focal_cross_entropy_loss = (modulating_factor * alpha_weight_factor * cross_entropy_loss)

        return keras.backend.mean(focal_cross_entropy_loss, axis=-1)

In [24]:
def multi_label_learning_demo(activation='sigmoid', loss=keras.backend.binary_crossentropy , mask_value=-1.0, is_FocalLoss_multilabel=True):
    """
    https://www.dlology.com/blog/how-to-multi-task-learning-with-missing-labels-in-keras/
    """
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.model_selection  import train_test_split
    import tensorflow as tf
    import numpy as np
    seed = 42
    np.random.seed(seed) # 乱数シード固定
    
    # 2D空間で100,000個のデータポイントをランダムに生成します。各軸の範囲は0〜1
    N =100000
    X = np.random.rand(N,2)
    
    # focal loss用に不均衡にする
    threshold = 0.05#0.5
    
    # place holder for Y
    Y = np.ones((N,3))
    if activation == 'sigmoid':
        print('sigmoid')
        Y[:,0] = X[:, 1] <= threshold
        Y[:,1] = X[:, 0] >= threshold
        Y[:,2] = X[:, 0] + X[:, 1] > 1
    else:
        print('not sigmoid')
        Y[:,0] = X[:, 1] <= threshold
        Y[:,1] = X[:, 1] > threshold
        Y[:,2] = 0

    # Mask for missing label.
    if mask_value is not None:
        # Drop 2% y0.
        Y[: int(N*0.020), 0] = mask_value
        # Drop 0.7% y1.
        Y[int(N*0.018): int(N*0.025), 1] = mask_value
        # Drop 1.1% y2.
        Y[int(N*0.024): int(N*0.035), 2] = mask_value

    print(Y.shape, Y)
    for i in range(3): 
        print(pd.Series(Y[:,i]).value_counts())
        
    X_train, X_dev, y_train, y_dev = train_test_split(X[:-3000], Y[:-3000], test_size=0.9, random_state=seed)

    keras.backend.clear_session()
    model = keras.models.Sequential()

    input_dim = X_train.shape[1]
    nb_classes = y_train.shape[1]
    print('input_dim, nb_classes:', input_dim, nb_classes)

    model.add(keras.layers.Dense(10, input_dim=input_dim, activation='relu', name='input'))
    model.add(keras.layers.Dense(20, activation='relu', name='fc1'))
    model.add(keras.layers.Dense(10, activation='relu', name='fc2'))
    model.add(keras.layers.Dense(nb_classes, activation=activation, name='output'))

    if mask_value is not None:
        loss = build_masked_loss(loss, mask_value=mask_value)
        
    if is_FocalLoss_multilabel:
        loss = FocalLoss_multilabel(loss_function=loss).compute_loss

    model.compile(loss=loss, optimizer='nadam', metrics=['accuracy'])

    history = model.fit(X_train, y_train
                        , validation_data=(X_dev,y_dev)
                        , epochs=300
                        , batch_size=500000
                        , verbose=0)

    print(model.metrics_names, model.evaluate(X[-3000:],Y[-3000:]))
    
    return model

#### sigmoid +  Focal_Lossなし
- うまくいってる

In [25]:
model = multi_label_learning_demo(mask_value=None, is_FocalLoss_multilabel=False)
x = np.random.rand(1,2)
print(x, model.predict(x))

sigmoid
(100000, 3) [[0. 1. 1.]
 [0. 1. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95109
0.0     4891
dtype: int64
1.0    50034
0.0    49966
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.23062710452079774, 0.91]
[[0.28258797 0.45867659]] [[0.0449715  0.93581486 0.32908934]]


#### sigmoid +  binary_Focal_Loss使用
- うまくいってて、精度上がってる

In [26]:
model = multi_label_learning_demo(mask_value=None)
x = np.random.rand(1,2)
print(x, model.predict(x))

sigmoid
(100000, 3) [[0. 1. 1.]
 [0. 1. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
0.0    95031
1.0     4969
dtype: int64
1.0    95109
0.0     4891
dtype: int64
1.0    50034
0.0    49966
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.013308002134164174, 0.91]
[[0.28258797 0.45867659]] [[0.12703249 0.7340522  0.20853841]]


#### sigmoid +  欠損ラベル作成して、maskありでbinary_Focal_Loss使用
- うまくいってる

In [27]:
model = multi_label_learning_demo(mask_value=-1.0)
x = np.random.rand(1,2)
print(x, model.predict(x))

sigmoid
(100000, 3) [[-1.  1.  1.]
 [-1.  1.  1.]
 [-1.  1.  0.]
 ...
 [ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  1.  0.]]
 0.0    93132
 1.0     4868
-1.0     2000
dtype: int64
 1.0    94440
 0.0     4859
-1.0      701
dtype: int64
 1.0    49495
 0.0    49405
-1.0     1100
dtype: int64
input_dim, nb_classes: 2 3
3000/1 [============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

['loss', 'accuracy'] [0.01357833850880464, 0.91]
[[0.28258797 0.45867659]] [[0.17821449 0.7863685  0.20343795]]


#### softmax +  欠損ラベル作成して、maskありでcategorical_Focal_Loss使用
- エラー

In [28]:
model = multi_label_learning_demo(activation='softmax'
                                  , loss=keras.backend.categorical_crossentropy
                                  , mask_value=-1.0)
x = np.random.rand(1,2)
print(x, model.predict(x))

not sigmoid
(100000, 3) [[-1.  1.  0.]
 [-1.  1.  0.]
 [-1.  1.  0.]
 ...
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]]
 0.0    93132
 1.0     4868
-1.0     2000
dtype: int64
 1.0    94365
 0.0     4934
-1.0      701
dtype: int64
 0.0    98900
-1.0     1100
dtype: int64
input_dim, nb_classes: 2 3


InvalidArgumentError:  Incompatible shapes: [9700] vs. [9700,3]
	 [[node loss/output_loss/mul_8 (defined at C:\Users\shingo\Anaconda3\envs\tfgpu20\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_127343]

Function call stack:
distributed_function
